In [1]:
%matplotlib inline

# Neural Network

使用`torch.nn`库构建神经网络
> 此处我们使用大名鼎鼎的 LeNet，结构为 :  
`32×32×1`-> `5×5*6conv`->`28×28×6`->`2×2pooling`->`14×14×6`->`5×5*16conv`->`10×10*16`->`2*2pooling`->`5×5*16`->`fc`->……

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

`nn.Conv2d`含有`__call__()`方法，因而可以将实例直接当做方法使用。

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
        # Max pooling函数中，如果核是正方形，那么可以只使用一个数字作为参数
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:] #除了batch dim之外的所有维度数，torch将一个batch里面的所有数据都合成一个tensor输入
        num_features = 1
        for s in size:
            num_features*=s
        return num_features

net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

backward 函数已经被自动创建。

In [4]:
name = list(net.named_parameters())

In [5]:
for i in name:
    print(i[0],i[1].size())

conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([16, 6, 5, 5])
conv2.bias torch.Size([16])
fc1.weight torch.Size([120, 400])
fc1.bias torch.Size([120])
fc2.weight torch.Size([84, 120])
fc2.bias torch.Size([84])
fc3.weight torch.Size([10, 84])
fc3.bias torch.Size([10])


举例解释上面的参数的含义：`16`个`5×5×6`的卷积核，每个卷积核将整`6`通道的图片卷积层`1`通道的图片，然后`16`个`1`通道的图片合在一起，变成`16`通道的图片、其他的单个向量为`bias`，每一个通道对应一个`bias`

In [6]:
input_matric = torch.randn(1,1,32,32) # 一个batch里面有1个32×32×1的图片

`torch.nn`只接受`batch`的输入，因而第一个维数一定是样本数。

In [7]:
out = net(input_matric)
print(out)

tensor([[ 0.2094, -0.0582,  0.1253, -0.0118,  0.0321,  0.0956,  0.0770, -0.0644,
         -0.0028, -0.0696]], grad_fn=<AddmmBackward>)


In [8]:
net.zero_grad()  # 将net 中的所有梯度设置为0 每个batch中都需要进行，因为torch中的梯度是积累的

In [9]:
out.backward(torch.randn(1,10)) #制定初始的梯度，瞎几把搞，就试一下而已

## loss function

In [10]:
out = net(input_matric)

In [11]:
target = torch.randn(10)  #随机值作为样例
target = target.view(1,-1) #使target和output的形状相同
criterion = nn.MSELoss()
loss = criterion(out, target) #对output 和 target进行MSE计算
print(loss)


tensor(1.3518, grad_fn=<MseLossBackward>)


In [12]:
print(loss.grad_fn)  #MSELoss的梯度类型
print(loss.grad_fn.next_functions[0][0]) #没弄懂，不着急
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #relu

## 对loss进行BP

In [13]:
net.zero_grad()

print(net.fc1.bias.grad)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [14]:
loss.backward()

In [15]:
print(net.fc1.bias.grad)

tensor([ 0.0000, -0.0267, -0.0192,  0.0000,  0.0000,  0.0054, -0.0204,  0.0000,
        -0.0055,  0.0068,  0.0513,  0.0083, -0.0090,  0.0000,  0.0000,  0.0000,
        -0.0208,  0.0000,  0.0000, -0.0118,  0.0000,  0.0000, -0.0095,  0.0000,
         0.0000,  0.0000, -0.0127,  0.0000,  0.0000, -0.0172, -0.0082,  0.0000,
         0.0000, -0.0093, -0.0131, -0.0276,  0.0070,  0.0000, -0.0057,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0069,  0.0000,  0.0000,  0.0000,  0.0131,
         0.0000,  0.0297,  0.0232,  0.0000, -0.0002,  0.0000,  0.0000,  0.0000,
        -0.0055, -0.0087,  0.0000,  0.0000,  0.0000,  0.0000,  0.0093, -0.0036,
         0.0000,  0.0139,  0.0123,  0.0000,  0.0015,  0.0249,  0.0000, -0.0401,
        -0.0180, -0.0107,  0.0000, -0.0126,  0.0012,  0.0000,  0.0000, -0.0011,
        -0.0134, -0.0204,  0.0000,  0.0196,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0140,  0.0000,  0.0000, -0.0102,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000, -0.0207,  0.0000,  0.00

## 更新权重

In [16]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(),lr=0.01)

optimizer.zero_grad()
out = net(input_matric)
loss = criterion(out, target)
loss.backward()

In [17]:
optimizer.step()  #执行一次更新

In [22]:
a = torch.ones(4,4)